# COMS 4995_002 Deep Learning Assignment 1
Due on Monday, Oct 9, 11:59pm

This assignment can be done in groups of at most 3 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Conrad De Peuter, cld2167

Member 2: Daniel First, df2450

Member 3: Arjun Singh, asd2171

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [372]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        
        eps = .01
        self.parameters = {'layer_dimensions':layer_dimensions, 'drop_prob':drop_prob, 'reg_lambda':reg_lambda} 
        self.num_layers = len(layer_dimensions)
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        
        # init parameters
        for l in range(1, self.num_layers):
            # xavier init
            self.parameters['W' + str(l)] = np.random.normal(size=(layer_dimensions[l], layer_dimensions[l-1]))  * np.sqrt(2)/np.sqrt(layer_dimensions[l-1])
            self.parameters['b' + str(l)]= np.ones((layer_dimensions[l], 1)) * eps
    
        # print all params:
        print(self.parameters.keys())
        
    def affineForward(self, A_, W_, b_):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
#         print("forward, shape W", W_.shape)
#         print("forward, shape A", A_.shape)
#         print("forward, shape b", b_.shape)
        return np.dot(W_, A_) + b_, [A_, W_, b_]
        

    def activationForward(self, A_, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        
        if activation == "relu":
            return self.relu(A_)
        if activation == "softmax":
            return self.softmax(A_)
        
        return A_

    def softmax(self, x):
        # softmax
        
        e_x = np.exp(x - np.max(x,axis=0))
        return e_x / e_x.sum( axis=0)
    
    def relu(self, X):
        return np.maximum(0,X)
            
    def dropout(self, A, prob):
        """
        :param A: 
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        
        # make a mask, binary mask
        # where that mask is 0, drop those activation units
        # where its one, scale them up by 1/(1-p)
        # in forward we need to apply this mask
        
        M = np.random.binomial([np.ones(A.shape)],1-prob)[0] 
        A = np.multiply(A * (1/(1-prob)), M)
        
        return A, M

    def forwardPropagation(self, X, training=False):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        cache = {}
        A = X
        
        for l in range(1, self.num_layers):
            
            layer_cache = {}
            layer_cache["linear"] = [A,  self.parameters["W" + str(l)], self.parameters["b" + str(l)]]
            Z, cache_l = self.affineForward(A, self.parameters["W" + str(l)], self.parameters["b" + str(l)])
            
            if l != self.num_layers - 1:
                if self.drop_prob > 0 and training:
                    Z, dropout_mask = self.dropout(Z, self.drop_prob)
                    layer_cache['dropout'] = dropout_mask
                        
                cache_a = self.activationForward(Z)
                
            else:
                cache_a = Z
            layer_cache['activation'] = cache_a
            cache[str(l)] = layer_cache
            
            A = cache_a
        
        return A, cache
    
    def costFunction(self, AL, y, reg):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """

        # softmax
        
        AL = self.softmax(AL)
        n_samples = AL.shape[1]
        true_labels = one_hot(y)
 
        y_hat = AL[y, range(y.shape[0])]
    
        # cost with regularization
        cost = -np.sum(np.log(y_hat))/ AL.shape[1] + (0.5)*self.reg_lambda * (1/n_samples) \
            * sum([scipy.linalg.norm(self.parameters["W" + str(l)],ord='fro') for l in range(1, self.num_layers)])
        
        if self.reg_lambda > 0:
            # add regularization
            pass
        
        # gradient of cost, should be same shape as input
        # 
        #dAL = -np.divide(true_labels, AL) 
        dAL = AL - true_labels
        return cost, dAL

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        A, W, b = cache['linear']
        n_samples = A.shape[1]
        #output_last_layer = cache[str(layer-1)]['activation']

     
        dW = (1/A.shape[1]) * np.dot(dA_prev,A.T) + self.reg_lambda * (1/n_samples) * W
        db = (1/A.shape[1]) * np.sum(dA_prev, axis=1, keepdims=True)
        
        dA_prev = np.dot(W.T,dA_prev)
        return dA_prev, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        
        zl = cache['activation']
        
        # dA is derivative last layer, first time it will be cost
        return np.multiply(dA , self.relu_derivative(zl, dA))  
    
    def relu_derivative(self, cached_x, dx):
        """
        Inputs:
        dx: upstream derivative
        cached_x: input of relu

        Returns:
        dx: gradient with respect to cached_x
        """     
        return 1.0 * (cached_x > 0)

    def dropout_backward(self, dA, cache):
        # in backwards we need to apply that mask to the derivatives
        # so cache here contains the dropout mask you used in forward prop
        # but also need to increase the nonzero elements by 1/(1-p)
        mask = cache['dropout']
        p = self.drop_prob
        return np.multiply((1/(1-p)) * mask, dA)

    def backPropagation(self, dAL, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {}
        dA_prev = dAL
        
        for l in range(self.num_layers-1, 0, -1):
            cached_l = cache[str(l)]
            
            if l != self.num_layers - 1:
                dA_prev = self.activationBackward(dA_prev, cached_l)
    
            # affine backwards
            if l != self.num_layers - 1 and self.drop_prob > 0:
                #call dropout_backward
                dA_prev = self.dropout_backward(dA_prev, cached_l)
          
                
            dA_prev, dW, db = self.affineBackward(dA_prev ,cached_l)
            gradients["dW" + str(l)] = dW
            gradients["db" + str(l)] = db
        
        if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
            pass
        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        for l in range(1, self.num_layers):
            #self.parameters["W" + str(l)] = self.parameters["W" + str(l)] - alpha * gradients["dW" + str(l)]
            self.parameters["W" + str(l)] = self.parameters["W" + str(l)] * (1 - alpha * self.reg_lambda/self.parameters["W" + str(1)].shape[0]) - alpha * gradients["dW" + str(l)]
            self.parameters["b" + str(l)] = self.parameters["b" + str(l)] - alpha * gradients["db" + str(l)]
            
    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        # get validation as last 10%
        validation_size = int(X.shape[1]*.1)
        orig_size = X.shape[1]
        validation_X = X[:,orig_size-validation_size:]
        validation_y = y[orig_size-validation_size:]
        
        # remove validation data from X and y
        X = X[:,0:orig_size-validation_size]
        y = y[0:orig_size-validation_size]

        print("X train shape:", X.shape)
        print("y train shape:", y.shape)
        print("X validation shape:", validation_X.shape)
        print("y validation shape:", validation_y.shape)
        
        
        for i in range(0, iters):
           
            batch_x, batch_y = self.get_batch(X, y, batch_size)
            
            # forward prop
            last_layer, cache = self.forwardPropagation(batch_x, training=True)
            
            # compute loss
            cost, cost_deriv = self.costFunction(last_layer, batch_y, self.reg_lambda)

            # compute gradients
            gradients = self.backPropagation(cost_deriv, cache)

            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)
            # gradually decrease alpha
            if i % 1000 == 0 and i != 0:
                alpha /= 2
                
            if i % print_every == 0:
                
                print("\n************")
                print("iteration","iter", i)
                print("COST", cost)
                print("alpha", alpha)
                print("************")
                
                # check train and validation accuracies
                forward_x = self.predict(batch_x)
                preds = np.argmax(forward_x, axis=0)
                print("train accuracy = ", np.mean(preds == batch_y))
                forward_validation = self.predict(validation_X)
                preds_y = np.argmax(forward_validation, axis=0)
                print("validation set accuracy = ", np.mean(preds_y == validation_y))
                
                # check gradients at each layer
                for d in range(self.num_layers-1, 0, -1):
                    print("abs gradients dW"+str(d),np.mean(np.abs(gradients["dW"+str(d)])))
                
    def predict(self, X_):
        """
        Make predictions for each sample
        """
        forward, _ = self.forwardPropagation(X_, training=False)
        return forward

    def get_batch(self, X_, y_, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        # are minibatches random
        #get random set
        #print("get batch shape", X_.shape, y_.shape)
        ix = np.random.choice(X_.shape[1], batch_size, replace=True)
        X_batch = X_[:,ix]
        y_batch = y_[ix]
        return X_batch, y_batch

In [248]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [315]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0

    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [5]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [375]:
layer_dimensions = [X_train.shape[0],600, 300, 10]  # .51
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=5000, alpha=.1, batch_size=500, print_every=200)

dict_keys(['layer_dimensions', 'drop_prob', 'reg_lambda', 'W1', 'b1', 'W2', 'b2', 'W3', 'b3'])
X train shape: (3072, 45000)
y train shape: (45000,)
X validation shape: (3072, 5000)
y validation shape: (5000,)

************
iteration iter 0
COST 2.66416219557
alpha 0.1
************
train accuracy =  0.104
validation set accuracy =  0.1004
abs gradients dW3 0.0184242767308
abs gradients dW2 0.00284796956897
abs gradients dW1 0.00300792723078

************
iteration iter 200
COST 1.82389703931
alpha 0.1
************
train accuracy =  0.354
validation set accuracy =  0.3624
abs gradients dW3 0.00321703891374
abs gradients dW2 0.000261164101146
abs gradients dW1 0.000309511779488

************
iteration iter 400
COST 1.89680433772
alpha 0.1
************
train accuracy =  0.322
validation set accuracy =  0.315
abs gradients dW3 0.00825915441505
abs gradients dW2 0.000642933177297
abs gradients dW1 0.000987667953997

************
iteration iter 600
COST 1.6234472418
alpha 0.1
************
tra

In [390]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-cld2167_df2450_asd2171', y_predicted)

In [356]:
print(y_predicted.shape)

(10, 10000)


In [378]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y)

[[-1.06249992  1.33005522  3.33509538 ..., -2.39236761 -0.86794965
   0.08415279]
 [-0.13822497  5.00114589 -1.96705173 ..., -3.40479755 -1.89508627
  -2.02070299]
 [ 1.87596489 -0.96333112  1.23368522 ...,  1.86587281  1.35867448
   0.83086348]
 ..., 
 [-3.17023337 -1.68197914 -2.52418477 ..., -0.11633853 -1.12995952
   2.56944033]
 [ 0.50728536  3.48634414  3.37555869 ..., -0.14947373 -2.32278338
  -1.93148091]
 [-3.78493452  5.53040632 -3.16095512 ..., -1.24449911 -1.7034949
  -1.49678932]]


## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [388]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=.1, reg_lambda=.5)
NN2.train(X_train, y_train, iters=5000, alpha=0.1, batch_size=500, print_every=200)

dict_keys(['layer_dimensions', 'drop_prob', 'reg_lambda', 'W1', 'b1', 'W2', 'b2', 'W3', 'b3'])
X train shape: (3072, 45000)
y train shape: (45000,)
X validation shape: (3072, 5000)
y validation shape: (5000,)

************
iteration iter 0
COST 2.58016331291
alpha 0.1
************
train accuracy =  0.088
validation set accuracy =  0.1186
abs gradients dW3 0.0174188873688
abs gradients dW2 0.00214214633315
abs gradients dW1 0.00217210268674

************
iteration iter 200
COST 1.97463809129
alpha 0.1
************
train accuracy =  0.28
validation set accuracy =  0.3184
abs gradients dW3 0.0049066775604
abs gradients dW2 0.000511439454163
abs gradients dW1 0.000778332690015

************
iteration iter 400
COST 1.80115679594
alpha 0.1
************
train accuracy =  0.376
validation set accuracy =  0.3876
abs gradients dW3 0.00419619824588
abs gradients dW2 0.000467325003419
abs gradients dW1 0.000670069537733

************
iteration iter 600
COST 1.74381903594
alpha 0.1
************
tra

In [391]:
y_predicted2 = NN2.predict(X_test)
save_predictions('ans2-cld2167_df2450_asd2171.npy', y_predicted2)